## Pickle Database

In [1]:
import pickle

In [2]:
data = [{"name": "Kumar shanu"}]

with open('users.pk', 'wb') as f:
    pickle.dump(data, f)


In [3]:
data_frm_pkl = pickle.load(open('users.pk', 'rb'))
data_frm_pkl

[{'name': 'Kumar shanu'}]

In [4]:
type(data_frm_pkl)

list

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(data)
df

,name
0,Kumar shanu


In [7]:
df.to_pickle('df_pkl')


In [8]:
df2 = pd.read_pickle('df_pkl')
df2

,name
0,Kumar shanu


In [9]:
df == df2

,name
0,True


## SQLAlchemy

In [10]:
class Movie:
    name = 'unknown'

    def __init__(self, name="", *args, **kwargs):
        self.name = name

In [11]:
movie_obj = Movie()
movie_obj.name

''

In [12]:
movie_obj.name = "Avengers"

In [13]:
movie_obj.name

'Avengers'

In [14]:
from dataclasses import dataclass

@dataclass
class Movie:
    name: str = "Unknown"
    genre: str = 'Action'
    year: int = None


In [15]:
movie_obj = Movie()
movie_obj.name

'Unknown'

In [16]:
movie_obj.genre

'Action'

In [17]:
movie_obj.name = 'Civil War'

In [18]:
movie_obj.name

'Civil War'

In [19]:
movie_obj.year

In [20]:
movie_obj.year = '2016'
movie_obj.year

'2016'

In [21]:
data =[ {
    "name": "Intersteller",
    "genre": "Sci-Fi"
},
{
    "name": "The Martian",
    "genre": "Sci-Fi"
},
{
    "name": "Arrival",
    "genre": "Sci-Fi"
}]

In [22]:
df = pd.DataFrame(data)
df

,name,genre
0,Intersteller,Sci-Fi
1,The Martian,Sci-Fi
2,Arrival,Sci-Fi


In [23]:
!pip install sqlalchemy

In [24]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [25]:
engine = create_engine('sqlite:///app.db')       #mysql, postgres

In [26]:
Session = sessionmaker(bind=engine)
mysess = Session()

In [27]:
class Movie:            # table
    name: str = "Unknown"   #col
    genre: str = 'Action'       #col
    year: int = None            #col


In [28]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()


In [29]:
from sqlalchemy import Column, Integer, String

class Movie(Base):            # table
    __tablename__ = "movies"

    id = Column(Integer, primary_key=True)
    name: str = Column(String)     # "Unknown"   #col
    genre: str = Column(String)    #'Action'       #col
    year: int = Column(Integer)                           #col

    def __repr__(self):
        return f"<Movie name={self.name}>"
        


In [30]:
# Add to database
Base.metadata.create_all(engine)

In [31]:
# add  row entry
movie_obj = Movie(name="intesteller", genre="Sci-Fi")
movie_obj.name

'intesteller'

In [32]:
movie_obj.id

In [33]:
mysess.add(movie_obj)   #prepare to save
mysess.commit()             # save

In [34]:
movie_obj.id

8

In [35]:
movie_obj2 = Movie(name="Avangers")
mysess.add(movie_obj2)
mysess.commit()
movie_obj2.id

9

In [36]:
print(movie_obj2.genre)

None


In [37]:
movie_obj2

<Movie name=Avangers>

CRUD in SQLAlchemy
- Create
- Retrieve
- Update
- Delete


In [38]:
Session = sessionmaker(bind=engine)
session = Session()

In [39]:
# create
movie = Movie(name="Gaurdians of the Galaxy")
session.add(movie)
session.commit()
movie.id

10

In [40]:
# retreive

# get 1 item
movie_a = session.query(Movie).get(5)
print(movie_a.id, movie_a.name)

5 Intersteller


In [41]:
# List
qs = session.query(Movie).all()
qs

[<Movie name=Avangers>,
 <Movie name=Gaurdians of the Galaxy>,
 <Movie name=Gaurdians of the Galaxy>,
 <Movie name=Intersteller>,
 <Movie name=Avangers>,
 <Movie name=Gaurdians of the Galaxy>,
 <Movie name=intesteller>,
 <Movie name=Avangers>,
 <Movie name=Gaurdians of the Galaxy>]

In [42]:
# List & Filter

qs = session.query(Movie).filter_by(name='Avangers').all()
qs

[<Movie name=Avangers>, <Movie name=Avangers>, <Movie name=Avangers>]

In [43]:
# List and Filter by column value containing something

qs = session.query(Movie).filter(Movie.name.contains('intes')).all()
qs

[<Movie name=intesteller>]

In [44]:
my_query = input("what are you looking for?") or 'Unknown'
qs = session.query(Movie).filter(Movie.name.contains(my_query)).all()
print(qs)

[]


In [45]:
# Update

movie_a = session.query(Movie).get(5)
movie_a.description = 'Movie'
print(movie_a.id, movie_a.name, movie_a.description)
session.commit()

5 Intersteller Movie


In [46]:
print(movie_a.id, movie_a.name, movie_a.description)

5 Intersteller Movie


In [47]:
qs = session.query(Movie).filter(Movie.name.contains('intes')).all()

for movie_obj in qs:
    movie_obj.name = "Intersteller"

session.commit()

In [49]:
# Delete
movie_a = session.query(Movie).get(2)
print(movie_a)

session.delete(movie_a)
session.commit()

<Movie name=Avangers>


In [57]:
session.flush()

In [61]:
movie_a = session.query(Movie).get(1)
print(movie_a)

None


## SQLAlchemy & Pandas

In [50]:
!pip install sqlacodegen

ERROR: Could not find a version that satisfies the requirement sqlacodegen (from versions: none)
ERROR: No matching distribution found for sqlacodegen


In [63]:
!sqlacodegen sqlite:///app2.db

/bin/bash: sqlacodegen: command not found


In [64]:
engine = create_engine('app2.db')
Session = sessionmaker(engine)
session = Session()

ArgumentError: Could not parse rfc1738 URL from string 'app2.db'

In [67]:
stored_data = movie_obj.__dict__
stored_data

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd6ffea6130>,
 'description': 'Movie'}

In [76]:
df = pd.read_sql_table('movies', engine)
df

,id,name,genre,year
0,2,Avangers,None,None
1,3,Gaurdians of the Galaxy,None,None
2,4,Gaurdians of the Galaxy,None,None
3,5,Intersteller,Sci-Fi,None
4,6,Avangers,None,None
5,7,Gaurdians of the Galaxy,None,None


In [77]:
df.to_sql(
    'movies',
    engine,
    dtype={
        "name": String,
        "genre": String,
        "year": Integer,
        "description": String
    }
)

ValueError: Table 'movies' already exists.

In [78]:
!sqlacodegen sqlite:///app.db

/bin/bash: sqlacodegen: command not found
